<a href="https://colab.research.google.com/github/NahuelRepetto/Programacion-Concurrente/blob/main/TrabajoN1Parte3Cpp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [196]:
%%writefile Servidor.cpp
#include<stdlib.h>
#include<stdio.h>
#include<string.h>
#include<iostream>
#include<fcntl.h>
#include<unistd.h>
#include<sys/stat.h>

using namespace std;

#define CANT_ELEMENTOS 4  

int calcularMaximo(int vec[])
{
  int max = vec[0];
	int comparacion;
	for(int i=0; i<CANT_ELEMENTOS; i++)
	{
		comparacion = vec[i];
		if(comparacion > max)
			max = vec[i];
	}
	return max;
}

int calcularMinimo(int vec[])
{
  int min = vec[0];
	int comparacion;
	for(int i=0; i<CANT_ELEMENTOS; i++)
	{
		comparacion = vec[i];
		if(comparacion < min)
			min = vec[i];
	}
	return min;
}

int calcularSuma(int vec[])
{
  int sum = 0;
	for(int i=0; i<CANT_ELEMENTOS; i++)
		sum += vec[i];
	return sum;
}

float calcularPromedio(int vec[])
{  
	return (float)calcularSuma(vec)/CANT_ELEMENTOS;
}


int main (int argc, char *argv[])
{
	int num[4];
	float retorno[4];

  //CREO TUBERIAS PARA RECIBIR DATOS
	if( mkfifo("./fifoA",0666) == -1 )
	{
		perror("Error al crear en mkfifo");
		return EXIT_FAILURE;	
	}

  //ABRO UNA TUBERIA EN MODO LECTURA
	int r = open("./fifoA", O_RDONLY);  
  
	//RECIBO DATOS
  if( read( r, num, sizeof( num ) ) == -1 )
	{
		perror("Error en read()");
		return EXIT_FAILURE;
	}	

	//PROCESO LOS DATOS
	retorno[0] = (float)calcularMinimo(num);
	retorno[1] = (float)calcularMaximo(num);
	retorno[2] = (float)calcularSuma(num);
	retorno[3] = calcularPromedio(num);

	//CREO TUBERIAS PARA ENVIAR DATOS
  if( mkfifo("./fifoB",0666) == -1 )
	{
		perror("Error al crear en mkfifo");
		return EXIT_FAILURE;	
	}
	
	//ABRO UNA TUBERIA EN MODO ESCRITURA
	int w = open("./fifoB", O_WRONLY);
	
	//ENVIO DATOS  
	if( write( w, retorno, sizeof( retorno ) ) == -1 )
	{
		perror("Error en write()");
		return EXIT_FAILURE;
	}	 
	
  close(r);  
	close(w);	
	unlink("./fifoA");
	unlink("./fifoB");
	return EXIT_SUCCESS;
}

Overwriting Servidor.cpp


In [321]:
%%writefile Cliente.cpp
#include<stdlib.h>
#include<stdio.h>
#include<string.h>
#include<sstream>
#include<iostream>
#include<fcntl.h>
#include<unistd.h>
#include<sys/stat.h>
#include<boost/lexical_cast.hpp>

using namespace std;

#define CANT_ELEMENTOS 4  

int main (int argc, char *argv[])
{		
	int num[4]; 
	float retorno[4];	

	//VERIFICO QUE SE HAYA ENVIADO LA CADENA CON LOS NUMEROS
  if( argc < 2 ) 
  {
    cout<<"Debe ingresar una cadena con una serie de cuatro números enteros no negativos separados con guión medio (-)"<<endl;
    exit(1);
  }

	//RECUPERO LOS NUMEROS DE LA CADENA ENVIADA	
	int ingreso = 0;
	string lectura;
	string cadena = argv[1];                  
  stringstream input_stringstream(cadena);                    
  while (getline(input_stringstream, lectura, '-'))
  {
		if(ingreso >= CANT_ELEMENTOS)	//PARA NO INGERSAR MAS DE LO PERMITIDO
		{
			cout << "Se enviaron mas valores de los esperados, ingrese maximo 4." << endl;
			exit(1);
		}

	  //PASO DE STRING A ENTERO  
		int i;   
    try 
		{			
      i = boost::lexical_cast<int>(lectura);        				
    }
    catch (boost::bad_lexical_cast const &e) //SI LA ENTRADA ES INCORRECTA (NO NUMERICA) O AL CONVERTIRLA EXCEDE EL RANGO DE INT
		{        
      cout << "Entrada invalida '" << lectura <<"', deben ingresarse solo numeros separados por guion medio (-)." << endl;
			exit(1);
    }
		num[ingreso] = i;		
		ingreso++;
  }

	//LA ABRO UNA EN MODO ESCRITURA
	int w = open("./fifoA", O_WRONLY);	
	
	//ENVIO DATOS 
	if( write( w, num, sizeof( num ) ) == -1 )
	{
		perror("Error en write()");
		return EXIT_FAILURE;
	}	

	//LA ABRO EN MODO LECTURA
	int r = open("./fifoB", O_RDONLY);   
	
	//RECIBO DATOS
  if( read( r, retorno, sizeof( retorno ) ) == -1 )
	{
		perror("Error en read()");
		return EXIT_FAILURE;
	}
	
	//MUESTRO LOS RESULTADOS
	cout << "Numeros recibidos: ";
	for(int i = 0; i < CANT_ELEMENTOS; i++)
		cout << num[i] << "   ";
		
	cout << endl << "Minimo: " << retorno[0];
	cout << endl << "Maximo: " << retorno[1];
	cout << endl << "Suma: " << retorno[2];
	cout << endl << "Promedio: " << retorno[3];	
	cout << endl << "Cantiad de numeros: " << CANT_ELEMENTOS;	
			
	close(w);	
	close(r);			
	return EXIT_SUCCESS;
}

Overwriting Cliente.cpp


In [197]:
!g++ -o Servidor Servidor.cpp

In [322]:
!g++ -o Cliente Cliente.cpp

In [323]:
!nohup ./Servidor 1>salida 2>/dev/null &

In [325]:
!./Cliente 45-5-3-4

Numeros recibidos: 45   5   3   4   
Minimo: 3
Maximo: 45
Suma: 57
Promedio: 14.25
Cantiad de numeros: 4

In [192]:
%%writefile cerrar.cpp
#include<stdlib.h>
#include<unistd.h>

int main (int argc, char *argv[])
{		
	unlink("./fifoA");
  unlink("./fifoB");		
	return EXIT_SUCCESS;
}

Overwriting cerrar.cpp


In [193]:
!g++ -o cerrar cerrar.cpp

In [201]:
!./cerrar

In [ ]:
!ls